# Simulating LJ chains using openMM

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
%%capture
!conda install -c conda-forge openmm mdtraj parmed
!pip install py3dmol 

In [ ]:
import openmm as mm
from   openmm import app
from   openmm.unit import * 


import parmed
import mdtraj as md
import py3Dmol
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Simulation parameters
temperature = 293.15 * kelvin
pressure = 1 * bar
mass = 39.948 * amu
sig = 3.419 * angstrom
eps = 117.8 * kelvin * BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

box_size = 150 * angstrom  
natom    = 199
cutoff   = 3 * sig
     

In [ ]:
s    = parmed.Structure()

for i in range(natom):

  s.add_atom(atom     =  parmed.Atom(name='A',  mass=mass), 
             resname  = "LJ", 
             resnum   =  1, 
             chain    = 'A')

s.positions = np.random.uniform(0, box_size/angstrom, (natom, 3))
s.save('lj.pdb', overwrite=True)

In [ ]:

print(s.topology)
s.to_dataframe().head()

In [ ]:

system = s.createSystem(removeCMMotion=False) 

box_vecs = 150 * angstrom *np.eye(3)
system.setDefaultPeriodicBoxVectors(*box_vecs )

system.removeForce(0) # parmed adds some extra forces that we dont need

E_lj  = '4*eps*((sig/r)^12-(sig/r)^6); sig=0.5*(sig1+sig2); eps=sqrt(eps1*eps2)'
force = mm.CustomNonbondedForce(E_lj)
force.addPerParticleParameter('sig') 
force.addPerParticleParameter('eps')

# Particles are assigned properties in the same order as they appear in the System object
for _ in range(natom): 
    force.addParticle([sig, eps])

# Set force cutoff parameters
force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
force.setCutoffDistance(3.0 * sig)       # set cutoff (truncation) distance at 3*sigma
force.setUseSwitchingFunction(True)      # use a smooth switching function to avoid force discontinuities at cutoff
force.setSwitchingDistance(2.5 * sig)    # turn on switch at 2.5*sigma

# Add force to system. System will now take ownership of the NonbondedForce object
force_index = system.addForce(force) 

In [ ]:


print('Check if force uses PBC: ', force.usesPeriodicBoundaryConditions() )

print('No particles: ', force.getNumParticles() )

print('per-particle parameters for particle-0: ', force.getParticleParameters(0))

print('Check Energy function: ', force.getEnergyFunction() )

In [ ]:

integrator =  mm.LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds) 

#system.addForce(mm.MonteCarloBarostat(pressure, temperature)) # NPT

simulation = app.Simulation(s.topology, system, integrator) 

simulation.context.setPositions(s.positions)

# - Minimize the energy
simulation.minimizeEnergy()

# - Initialize velocities with random values at 300K.
simulation.context.setVelocitiesToTemperature(300)


# Reporters
simulation.reporters = []
simulation.reporters.append(app.DCDReporter('ljtraj.dcd', 100))
simulation.reporters.append(app.PDBReporter('ljtraj.pdb', 100))
simulation.reporters.append(app.StateDataReporter("ljscalars.csv", 10, 
                                              time=True, 
                                              potentialEnergy=True, 
                                              totalEnergy=True, 
                                              temperature=True, 
                                              volume=True))

simulation.step(30000)

In [ ]:
# Animation
view = py3Dmol.view(width=800, height=800)

view.addModelsAsFrames(open('ljtraj.pdb', 'r').read(),'pdb')
view.setBackgroundColor('white')
view.setStyle({'sphere': {'color':'green'}}) 

#
view.zoomTo()
view.animate({'loop': "forward"})
view.show()